In [ ]:
#from http://www.mikaelbrunila.fi/2017/03/27/scraping-extracting-mapping-geodata-twitter/
# https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/

import tweepy
from tweepy import OAuthHandler
import json
import re
import collections
import json_lines
import csv

consumer_key = '93WP2pKvICGqCvrdA6FWMbFLB'
consumer_secret = 'LWcPKp3oqEmYTTbVaNpRD5tgqLl2KAfe2bNGsBk5xwTjxLxqHH'
access_token = '3080541417-QQ15kHSfkZYBWWSqLHwKwfqB3INeYgxpY3P9IbO'
access_secret = '2nuMVxvX8TzMyl1xHkUvElNy6BqWjqdoolTvLLxwoXHi1'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [ ]:
places = api.geo_search(query="USA", granularity="country")
place_id = places[0].id

In [ ]:
tweets = api.search(q="place:%s" % place_id)
for tweet in tweets:
    print(tweet.text + " | " + tweet.place.name if tweet.place else "Undefined place")

In [ ]:
def barbieri_clean_up(tweet,original = True):
    #casefold() lowercases all words. Replace corrects for emojis that are not separated by space to words.
    if original:
        raw = tweet.text.casefold().encode(encoding = 'unicode-escape').decode().replace('\\U',' \\U').split()
    else:
        raw = tweet.casefold().encode(encoding = 'unicode-escape').decode().replace('\\U',' \\U').split()
    #remove all hyperlinks
    new_raw = []
    for item in raw:
        if item.find('http') == -1:
            if item.find('#') == -1:
                if item.find('@') == -1:
                    new_raw.append(item)
    return ' '.join(new_raw)

def emoji_count(clean_tweet):
    i = 0
    for item in clean_tweet:
        if item.find('\\U') > -1:
            i += 1
    return i

In [ ]:
[print(tweet.text) for tweet in tweets]
print()
for tweet in tweets:
    clean_tweet = barbieri_clean_up(tweet)
    print(clean_tweet)
    print()
    print(emoji_count(clean_tweet))
    print()

In [ ]:
folder = './data/tweet_id_files/twitter-events-2012-2016/'
paths = [folder + '2012-euro2012.jsonl',folder + '2012-mexican-election.jsonl',folder + '2012-superbowl.jsonl',
         folder + '2012-sxsw.jsonl']
regex = re.compile(r'\\d+(.*?)(?:\\u263a|\\U0001f645)')
counts = collections.Counter()
filtered_tweets_text = []
with open("output_1.csv", "w") as csv:
    for path in paths:
        print(path)
        f = open(path,'rb')
        for line in json_lines.reader(f):
            tweet = line
            if 'metadata' in tweet:
                l = tweet.metadata['iso_language_code']
            else:
                l = tweet['lang']
            if l == 'en':
                if 'full_text' in tweet:
                    text = tweet['full_text']
                else:
                    text = tweet['text']
                e = re.findall(u'[\\U0001f600-\\U0001f650]', text)
                if len(e)>0:
                    clean = barbieri_clean_up(text,original = False)
                    filtered_tweets_text += [clean]
                    row = clean + '\n'
                    csv.write(row)
                    for char in e:
                        counts[char] += 1
        f.close()
          
    for char, count in counts.most_common():
        print("%s %5i" % (char, count))